In [259]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
import matplotlib.dates as mdates

Data Import

In [260]:
df1 = pd.read_csv('5 memes.csv', header=None, names=['Week', 'Skibidi Toilet', 'Chill Guy', 'Sigma Boy', 'Rizz', 'Ohio Meme']) 
df2 = pd.read_csv('1 meme.csv', header=None, names=['Week', 'Very Mindful, Very Demure'])

Merging data

In [261]:
df1 = df1.drop([0, 1])
df2 = df2.drop([0, 1])
df1.reset_index(drop=True, inplace=True)
df2.reset_index(drop=True, inplace=True)
df1.shape, df2.shape

((262, 6), (262, 2))

In [262]:
df1['Very Mindful, Very Demure'] = df2['Very Mindful, Very Demure']
df = df1
del df1
del df2

Cleaning

In [263]:
df['Week'] = pd.to_datetime(df['Week'], format='%Y-%m-%d')

In [264]:
df = df[df['Week'] >= '2022-01-01']

In [265]:
df.shape

(179, 7)

In [266]:
df.head()

,Week,Skibidi Toilet,Chill Guy,Sigma Boy,Rizz,Ohio Meme,"Very Mindful, Very Demure"
83,2022-01-02,0,<1,0,1,1,0
84,2022-01-09,0,<1,0,1,1,0
85,2022-01-16,0,<1,0,1,1,0
86,2022-01-23,0,<1,0,1,1,0
87,2022-01-30,0,<1,0,1,1,0


In [267]:
df = df.replace("<1", 0)

In [268]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 179 entries, 83 to 261
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Week                       179 non-null    datetime64[ns]
 1   Skibidi Toilet             179 non-null    object        
 2   Chill Guy                  179 non-null    object        
 3   Sigma Boy                  179 non-null    object        
 4   Rizz                       179 non-null    object        
 5   Ohio Meme                  179 non-null    object        
 6   Very Mindful, Very Demure  179 non-null    object        
dtypes: datetime64[ns](1), object(6)
memory usage: 11.2+ KB


In [269]:
col = ['Skibidi Toilet', 'Chill Guy', 'Sigma Boy', 'Rizz', 'Ohio Meme',
       'Very Mindful, Very Demure']
df[col] = df[col].astype(int)

In [270]:
order = ['Week', 'Ohio Meme', 'Rizz', 'Skibidi Toilet', 'Chill Guy', 'Sigma Boy', 'Very Mindful, Very Demure']
df = df[order]

In [271]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 179 entries, 83 to 261
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Week                       179 non-null    datetime64[ns]
 1   Ohio Meme                  179 non-null    int32         
 2   Rizz                       179 non-null    int32         
 3   Skibidi Toilet             179 non-null    int32         
 4   Chill Guy                  179 non-null    int32         
 5   Sigma Boy                  179 non-null    int32         
 6   Very Mindful, Very Demure  179 non-null    int32         
dtypes: datetime64[ns](1), int32(6)
memory usage: 7.0 KB


Set up colors

In [272]:
color_map = {
    'Skibidi Toilet': '#636efa',
    'Chill Guy': '#ef553b',
    'Sigma Boy': '#00cc96',
    'Rizz': '#ab63fa',
    'Ohio Meme': '#ffa15a',
    'Very Mindful, Very Demure': '#19d3f3'
}

In [273]:
fig_line = px.line(df, x='Week', y=col, title='Average Meme Popularity Over Time',
               labels={'value': 'Popularity', 'variable': 'Meme'},
               template='plotly_dark', width=1300)

In [285]:
fig_line.show()

In [275]:
threshold = 5
df_long = df.melt(id_vars='Week', var_name='Meme', value_name='Popularity')
df_long = df_long[df_long['Popularity'] >= threshold]
df_long = df_long.sort_values(['Meme', "Week"])

In [276]:
periods = []
for meme, group in df_long.groupby('Meme'):
    group = group.reset_index(drop=True)
    start = group.iloc[0]['Week']
    for i in range(1, len(group)):
        if (group.iloc[i]['Week'] - group.iloc[i-1]['Week']).days > 7:
            periods.append((meme, start, group.iloc[i-1]['Week']))
            start = group.iloc[i]['Week']
    end = group.iloc[len(group)-1]['Week']
    periods.append((meme, start, end))
periods_df = pd.DataFrame(periods, columns=['Meme', 'Start', 'End'])

In [277]:
# Desired order
meme_order = ['Ohio Meme', 'Rizz', 'Skibidi Toilet', 'Chill Guy', 'Sigma Boy', 'Very Mindful, Very Demure']

# Set 'Meme' as categorical with that order
periods_df['Meme'] = pd.Categorical(periods_df['Meme'], categories=meme_order, ordered=True)

# Sort by this order
periods_df = periods_df.sort_values('Meme')


In [278]:
fig_gantt = px.timeline(periods_df, x_start='Start', x_end='End', y='Meme',
                        color='Meme',
                        color_discrete_map=color_map,
                        title='Meme Popularity Periods',
                        labels={'Meme': 'Meme', 'Start': 'Start Date', 'End': 'End Date'},
                        template='plotly_dark', width=1300)
fig_gantt.show()

In [279]:
total_search = df[col].sum().sort_values(ascending=False)
fig_bar = px.bar(total_search, x=total_search.index, y=total_search, color=total_search.index,
                color_discrete_map=color_map,
                title='Total Search Popularity of Memes',
                labels={'value': 'Total Popularity', 'index': 'Meme'},
                template='plotly_dark', width=1000)
fig_bar.show()

In [280]:
df['Year'] = df['Week'].dt.year

In [281]:
df_2022 = df[df['Year'] == 2022]
df_2023 = df[df['Year'] == 2023]
df_2024 = df[df['Year'] == 2024]
df_2025 = df[df['Year'] == 2025]

In [284]:
years = [df_2022, df_2023, df_2024, df_2025]
for year_df in years:
    yearly_total = year_df[col].sum().sort_values(ascending=False) 
    yearly_total = yearly_total.reset_index()
    yearly_total.columns = ['Meme', 'Total Popularity']

    fig_year = px.pie(
        yearly_total,
        values='Total Popularity',
        names='Meme',
        color='Meme',  # ✅ Tells Plotly to apply the color mapping
        color_discrete_map=color_map,
        title=f'Total Search Popularity of Memes in {year_df["Year"].iloc[0]}',
        template='plotly_dark',
        width=600
    )

    fig_year.show()